# Partie 4. - Classification documentaire

La classification documentaire consiste à catégoriser un texte ou un document dans une ou plusieurs catégorie prédéfinie. 

Les principes généraux sont les mêmes que ceux vus pour la classification d'images, à la différence que les données d'entrée ne sont pas des images mais du texte.

Il existe de nombreux algorithmes de classification de documents. Cependant, ceux qui fonctionnent le mieux à ce jour sont les systèmes reposant sur une représentation vectorielle des documents car celle-ci _encode_ le sens des informations et évite les écueils liés aux grandes variabilités syntaxiques que l'on retrouve dans les textes.

Pour cette partie, nous allons utiliser un modèle pré-entraîné de type BERT. 

Nouveauté par rapport au cours, nous allons étudier un modèle de classification _zero shot_ qui permet de spécifier les étiquettes lors de la phase d'inférence et non plus à l'entraînement. Ce nouveau type de modèle facilite grandement la mise en place d'un nouveau classifieur. La contre partie est des performances moindre.

## Import des bibliothèques logicielles et configuration

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification # Tokenizer & classification de textes à partir de transformers.
from transformers import pipeline # Mise en oeuvre de chaînes de traitement.
from datasets import load_dataset # Intégration de jeux de données.

from tqdm.notebook import tqdm
import ipywidgets as widgets

In [ ]:
# Configuration
PRE_TRAINED_MODEL_NAME = "BaptisteDoyen/camembert-base-xnli"

## 1. Chargement du jeu de données.

Le cas d'usage que nous allons résoudre est la classification de d'opinions d'avis utilisateur sur des livres.

Pour ce TP, l'opinion est binaire : **positif** ou **négatif**.

Première étape : récupération du jeu de données. Pour cela, exécuter la commande suivante qui permettre de télécharger les données depuis Internet.

In [ ]:
french_book_review_dataset = load_dataset("Abirate/french_book_reviews")

## 2. Chargement du modèle pré-entraîné et configuration

Ensuite, chargeons un modèle pré-entraîné à la classification _zero shot_. Le modèle que nous utilisons est un modèle ayant été appris sur la partie française du jeu de donées _XNLI_ (https://github.com/facebookresearch/XNLI).

In [ ]:
print("Loading or downloading model {}".format(PRE_TRAINED_MODEL_NAME))
tokenizer = AutoTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME) # Chargement du tokenizer.
model = AutoModelForSequenceClassification.from_pretrained(PRE_TRAINED_MODEL_NAME) # Chargement du modèle pour la classification de séquences de textes.
classifier = pipeline("zero-shot-classification", model=PRE_TRAINED_MODEL_NAME) # Réalisation de la chaîne de classification

Maintenant que le modèle est chargé, spécifions les catégories (_labels_) dans lesquels nous allons classer nos documents : **positif**, **négatif**.

Nous spécifions aussi une hypothèse pour mieux catégoriser le document. L'idée est d'estimer la probabilité conditionnelle `P(label|hypothèse)` pour chaque texte puis de sélectionner le label qui maximise cette probabilité.

In [ ]:
labels = ["négatif", "positif"] # catégories.
hypothesis_template = "L'avis sur ce livre est plutôt {}." # Hypothèse.

Affichage du format du dataset.

In [ ]:
total = len(french_book_review_dataset.data["train"])
print(french_book_review_dataset)

## 3. Embeddings

Afin d'être traité efficacement, le texte est transformé en vecteurs pour encoder le sens des informations. Ces vecteurs sont appelés des **embeddings**.

Ci-dessous, nous visualisons un de ces vecteurs. Il n'est pas compréhensible directement par un humain mais il l'est par la machine qui peut l'exploiter pour réaliser différentes tâches !

In [ ]:
pipeline_vector = pipeline('feature-extraction', model=PRE_TRAINED_MODEL_NAME)
embedding = pipeline_vector("Ceci est un texte d'exemple !")
print("Embedding du mot 'Ceci':")
print("")
print(embedding[0][0])

## 3. Evaluation du classifieur

Pour évaluer notre classifieur, nous allons utiliser la métrique simple de la précision : `nombre de bonnes prédictions / # total de prédictions`.

Il est aussi possible d'utiliser d'autres métriques comme le score `F1` ou d'autres que vous avez vu aux TPs précédents.

In [ ]:
def evaluate_classifier(dataset, model):
    """
    Fonction prenant en paramètre le jeu de données et un modèle et retournant la précision des prédictions.
    """
    predictions, groudtruth = [], []
    correct_classification_count = 0
    accuracy = 0.0
    for text, label in tqdm(zip(dataset.data["train"]["reader_review"], dataset.data["train"]["label"]), total=len(dataset.data["train"])):
        if(text.as_py() is None or text.as_py() == ""):
            continue
        
        prediction = model(text.as_py(), labels, multi_label=False, hypothesis_template=hypothesis_template)
        predicted_label = prediction["labels"][0]
        predicted_label_int = 1 if predicted_label == "positif" else 0
        predictions.append(predicted_label)
        groudtruth.append(label)
        if (predicted_label_int == label.as_py()):
            correct_classification_count += 1
        accuracy = correct_classification_count / len(predictions)
        if(len(predictions) % 100 == 0):
            print("Accuracy after {} texts processed = {}".format(len(predictions), accuracy))
    return accuracy

Lançons la classification sur l'ensemble des documents. Toutes les 100 prédictions, la fonction `evaluate_classifier` retourne la précision. 

Si le temps de traitement est trop long, vous pouvez arrêter le processus.

In [ ]:

accuracy = evaluate_classifier(french_book_review_dataset, classifier)
print(accuracy)

Les résultats sont encourageants, d'autant qu'aucune phase d'apprentissage n'a été réalisé préalablement.

L'avantage de cette approche est qu'elle permet de définir beaucoup plus rapidement de nouvelles chaînes de classification.

## 4. Test sur des textes écrits par vous !

Exécute le bloc de code ci-dessous pour saisir un texte. Le programme vous indiquera si ce qui est écrit est positif ou négatif !

In [ ]:
user_input_widget = widgets.Text( description='Votre avis?' )
output_widget = widgets.Label(value='En attente...' )

vb=widgets.VBox([user_input_widget, output_widget])


def callback(wdgt):
    prediction = classifier(user_input_widget.value, labels, multi_label=False, hypothesis_template=hypothesis_template)
    predicted_label = prediction["labels"][0]
    score = prediction["scores"][0]
    output_widget.value="{} avec un score de {}".format(predicted_label, score)


user_input_widget.on_submit(callback)

display(vb)

## 5. Création d'un nouveau classifieur

**Exercice** : modifier le code ci-dessous pour classer les documents selons les catégories suivantes : `subjectif`, `objectif`.

In [ ]:
book_category_label = ["objectif", "subjectif"] # TODO
book_category_hypothesis_template = "L'avis sur ce livre est {}." # TODO

user_input_widget = widgets.Text( description='Votre avis?' )
output_widget = widgets.Label(value='En attente...' )

vb=widgets.VBox([user_input_widget, output_widget])


def callback(wdgt):
    prediction = classifier(user_input_widget.value, book_category_label, multi_label=False, hypothesis_template=book_category_hypothesis_template)
    predicted_label = prediction["labels"][0]
    score = prediction["scores"][0]
    output_widget.value="{} avec un score de {}".format(predicted_label, score)


user_input_widget.on_submit(callback)

display(vb)


## Conclusion

Dans cette partie, nous avons vu la classification de documents. Point intéressant grâce aux récentes avancées scientifiques du domaine : il est possible de fournir les étiquettes à l'inférence !